In [17]:
import sys
sys.path.append("/Users/julietnwagwuume-ezeoke/_UILCode/gqe-phd/geomeppy/_scripts")

from typing import Union
from dataclasses import dataclass

from copy import deepcopy
import json

from case_edits.ezcase import EzCase

from recipes.amber import input
from recipes.exp_01 import input as input1, ns_axis
from geometry.wall_normal import WallNormal
from helpers.helpers import key_from_value
from methods.subsurfaces.pairs import DEFAULT_WINDOW, SubsurfacePair, DEFAULT_DOOR


from gplan.subsurfaces import SubsurfaceTranslator

from collections import Counter

In [ ]:
ez = EzCase(input, RUN_CASE=False)
ez

In [19]:
c = Counter()
zones_w_subsurfaces = []
for s in ez.case.geometry.subsurfaces.values():
    zones_w_subsurfaces.append(s.wall.zone)
    c[s]+=1

In [21]:
c.total()

18

In [29]:
ez.case.geometry.subsurfaces["Door_B_07_West_W03_2"].name

'Block 07 Storey 0 Wall 0003_2 Door'

In [68]:
# ez.case.idf.idfobjects["DOOR:INTERZONE"]
def get_partner_zone(interzone_door_name):
    door_obj = ez.case.idf.getobject("DOOR:INTERZONE", interzone_door_name)
    assert door_obj
    partner = door_obj["Outside_Boundary_Condition_Object"]
    partner_obj = ez.case.idf.getobject("DOOR:INTERZONE", partner)
    assert partner_obj
    partner_wall = partner_obj["Building_Surface_Name"]

    [wall] =[w for w in ez.case.geometry.walls.values() if w.name == partner_wall]

    return wall.zone, partner_wall



In [69]:
get_partner_zone("Block 07 Storey 0 Wall 0003_2 Door")

(Zone(Block 09), 'Block 09 Storey 0 Wall 0001_2')

In [73]:
for k, v in ez.case.geometry.subsurfaces.items():
    print(k)

Window_B_00_North_W02
Door_B_00_East_W01_1
Window_B_01_South_W04
Door_B_01_East_W01_1
Window_B_02_North_W02
Door_B_02_East_W01_1
Window_B_03_North_W02
Door_B_03_West_W03_3
Door_B_03_South_W04_1
Door_B_05_West_W03_1
Window_B_06_South_W04
Door_B_06_West_W03_2
Door_B_06_East_W01_3
Door_B_07_West_W03_2
Door_B_07_North_W02_1
Window_B_08_South_W04
Window_B_09_South_W04
Window_B_010_North_W02


In [90]:
c = Counter()
for k, v in ez.case.geometry.subsurfaces.items():
    c[v.wall.zone]+=1
    # try:
    if "Door" in k:
        # print(k, v.name)
        zone, p_wall = get_partner_zone(v.name)
        c[zone]+=1
        print(f"{k} has a partner. it is on {p_wall}")
    # except:
    #     print(f"no partner for {k}")


Door_B_00_East_W01_1 has a partner. it is on Block 07 Storey 0 Wall 0003_1
Door_B_01_East_W01_1 has a partner. it is on Block 06 Storey 0 Wall 0003_1
Door_B_02_East_W01_1 has a partner. it is on Block 03 Storey 0 Wall 0003_1
Door_B_03_West_W03_3 has a partner. it is on Block 06 Storey 0 Wall 0001_1
Door_B_03_South_W04_1 has a partner. it is on Block 04 Storey 0 Wall 0002_1
Door_B_05_West_W03_1 has a partner. it is on Block 07 Storey 0 Wall 0001_1
Door_B_06_West_W03_2 has a partner. it is on Block 07 Storey 0 Wall 0001_2
Door_B_06_East_W01_3 has a partner. it is on Block 08 Storey 0 Wall 0003_1
Door_B_07_West_W03_2 has a partner. it is on Block 09 Storey 0 Wall 0001_2
Door_B_07_North_W02_1 has a partner. it is on Block 010 Storey 0 Wall 0004_1


In [80]:
c

Counter({Zone(Block 07): 5,
         Zone(Block 06): 5,
         Zone(Block 03): 4,
         Zone(Block 00): 2,
         Zone(Block 01): 2,
         Zone(Block 02): 2,
         Zone(Block 08): 2,
         Zone(Block 09): 2,
         Zone(Block 010): 2,
         Zone(Block 04): 1,
         Zone(Block 05): 1})

In [81]:
[k for k,v in c.items() if v >=2]

[Zone(Block 00),
 Zone(Block 07),
 Zone(Block 01),
 Zone(Block 06),
 Zone(Block 02),
 Zone(Block 03),
 Zone(Block 08),
 Zone(Block 09),
 Zone(Block 010)]

In [91]:
ineligible_zones = [k for k,v in c.items() if v < 2]
print(ineligible_zones)
ineligible_surfaces = []
c = Counter()
for k, v in ez.case.geometry.subsurfaces.items():
    zone = v.wall.zone
    partner_zone = None
    if "Door" in k:
        partner_zone, p_wall = get_partner_zone(v.name)
    if zone in ineligible_zones or partner_zone in ineligible_zones:
        ineligible_surfaces.append(k)


[Zone(Block 04), Zone(Block 05)]


In [92]:
ineligible_surfaces

['Door_B_03_South_W04_1', 'Door_B_05_West_W03_1']

In [84]:

[z.zone_subsurfaces for z in ineligible_zones]

[[], [Subsurface(Door on Block 05 - West - W03_1)]]

In [53]:
ez.case.idf.getobject("DOOR:INTERZONE", "Block 07 Storey 0 Wall 0003_2 Door")


DOOR:INTERZONE,
    Block 07 Storey 0 Wall 0003_2 Door,    !- Name
    Project Door,             !- Construction Name
    Block 07 Storey 0 Wall 0003_2,    !- Building Surface Name
    Block 07 Storey 0 Wall 0003_2 Door Partner,    !- Outside Boundary Condition Object
    1.0,                      !- Multiplier
    21.650000000000006,       !- Starting X Coordinate
    1,                        !- Starting Z Coordinate
    17.7,                     !- Length
    7.2;                      !- Height

In [49]:
ez.case.geometry.walls["B_00_East_W01_1"].name

'Block 00 Storey 0 Wall 0001_1'

In [52]:
[wall] =[w for w in ez.case.geometry.walls.values() if w.name == 'Block 09 Storey 0 Wall 0001_2']
wall.zone

Zone(Block 09)

In [22]:
c = Counter()
for k,v  in ez.case.geometry.zones.items():
    if len(v.zone_subsurfaces) >= 2:
        print(v)
        c[v]+=1

Zone(Block 00)
Zone(Block 01)
Zone(Block 02)
Zone(Block 03)
Zone(Block 06)
Zone(Block 07)


In [23]:
c.total()

6

In [16]:
len(ez.case.geometry.zones)

11

In [5]:
zones_w_subsurfaces

[Zone(Block 00),
 Zone(Block 00),
 Zone(Block 01),
 Zone(Block 01),
 Zone(Block 02),
 Zone(Block 02),
 Zone(Block 03),
 Zone(Block 03),
 Zone(Block 03),
 Zone(Block 05),
 Zone(Block 06),
 Zone(Block 06),
 Zone(Block 06),
 Zone(Block 07),
 Zone(Block 07),
 Zone(Block 08),
 Zone(Block 09),
 Zone(Block 010)]